In [1]:
from PIL import Image
from skimage.feature import hog
import matplotlib.pyplot as plt
import numpy as np
import math
import sys
import cv2
import os
import time
import pandas as pd
from src.utils import Region, get_loss
from src.kcf.utils import *
from src.kcf.kcf import KCFtracker

In [2]:
import src.kcf.kcf as kcf
import src.kcf.utils as utils

In [3]:
import imp
imp.reload(kcf)

<module 'src.kcf.kcf' from 'C:\\Users\\xsyso\\OneDrive\\DataScience\\DSGA300108\\project\\Single Object Tracking\\src\\kcf\\kcf.py'>

In [2]:
def kcf_object_tracking(all_imgs, regions):
    kcf = KCFtracker(image=all_imgs[0], region=regions[0], cell_size=4, padding=1.5, feature='hog')
    losses = []
    for i, img in enumerate(all_imgs[1:]):
        
        tracked_region = kcf.track(img)
        loss = get_loss(regions[i+1], tracked_region)
        losses.append(loss)
        
    return losses

In [3]:
data_names = [tmp for tmp in os.listdir('./data/TinyTLP/') if not tmp.endswith('zip')]
data_names[:5], len(data_names)

(['Alladin', 'Aquarium1', 'Aquarium2', 'Badminton1', 'Badminton2'], 50)

In [6]:
video_name = data_names[8]
data_path = './data/TinyTLP/{}'.format(video_name)
ground_truth = pd.read_csv(data_path+'/groundtruth_rect.txt',header=None).values[:, 1:-1]
ground_truth = [Region(list(rect), data_mode='tlp', region_mode='square') for rect in ground_truth]

In [7]:
data_path

'./data/TinyTLP/Billiards1'

In [8]:
file_path_image = '{}/img'.format(data_path)
all_imgs_path = []
for file_name in os.listdir(file_path_image):
    all_imgs_path.append(os.path.join(file_path_image, file_name))
all_imgs_path = sorted(all_imgs_path)[:10]
all_imgs = [np.array(Image.open(img_path))/255 for img_path in all_imgs_path]

In [9]:
kcf_object_tracking(all_imgs, ground_truth)

0 1.0
1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [10]:
imp.reload(utils)
xxx = utils.get_subwindow(all_imgs[0], [287.5, 718.5], [67, 67], 4, 'hog')

In [11]:
from skimage.feature import hog

In [12]:
hog(all_imgs[0][100:110, 100:110, 0], pixels_per_cell=(4, 4), transform_sqrt=True, 
                 visualise=True, block_norm='L2-Hys', feature_vector=False)[1].squeeze().shape

(10, 10)

In [13]:
cv2.HOGDescriptor()

<HOGDescriptor 000001E3452B9F70>

In [42]:
15876 / 4 / 9 / 21

21.0

In [31]:
# regions = ground_truth
# kcf = kcf.KCFtracker(image=all_imgs[0], region=regions[0], cell_size=4, padding=1.5, feature='hog')
# losses = []
# for i, img in enumerate(all_imgs[1:]):
#     tracked_region = kcf.track(img)
#     loss = get_loss(regions[i+1], tracked_region)
#     losses.append(loss)

In [5]:
losses_all = []
with open('kcf.log', 'w') as f:
    for i, video_name in enumerate(data_names):
        data_path = './data/TinyTLP/{}'.format(video_name)
        ground_truth = pd.read_csv(data_path+'/groundtruth_rect.txt',header=None).values[:, 1:-1]
        ground_truth = [Region(list(rect), data_mode='tlp', region_mode='square') for rect in ground_truth]
        
        file_path_image = '{}/img'.format(data_path)
        all_imgs_path = []
        for file_name in os.listdir(file_path_image):
            all_imgs_path.append(os.path.join(file_path_image, file_name))
            
        all_imgs_path = sorted(all_imgs_path)
        all_imgs = [np.array(Image.open(img_path))/255 for img_path in all_imgs_path]

        losses = kcf_object_tracking(all_imgs, ground_truth)

        losses_all.append(losses)
        print(video_name, np.mean(losses))
        to_log = [video_name] + [str(np.mean(losses))] + [str(loss) for loss in losses]
        f.write('\t'.join(to_log) + '\n')

Alladin 52.27820553811296
Aquarium1 99.02169699844008
Aquarium2 674.613739469858
Badminton1 294.1790511600719
Badminton2 105.02677732357922
Basketball 131.3365243056705
Bharatanatyam 366.3704035740947
Bike 74.58484187584662
Billiards1 81.7212341834738
Billiards2 216.4143432479317
Boat 42.90623425539693
Boxing1 232.6361568257885
Boxing2 167.0853428151791
Boxing3 232.83617500351627
BreakfastClub 29.030713647628822
CarChase1 216.25501471410257
CarChase2 20.509802332499103
CarChase3 55.92987044234805
Dashcam 158.94472860891838
DriftCar1 395.80966089768003
DriftCar2 347.3765880262564
Drone1 114.80878053039741
Drone2 19.1870238508698
Drone3 81.90940579862168
Elephants 588.1882674446701
Helicopter 235.5714158900697
Hideaway 116.60570676805281
IceSkating 77.19503998103062
ISS 358.3377472528894
Jet1 163.70737935269818
Jet2 34.084290794119646
Jet3 47.895241871442344
Jet4 128.6053900214256
Jet5 169.3663868219733
KinBall1 169.23549689191546
KinBall2 12.180506016592139
KinBall3 286.7328274248111
Li